In [14]:
from s3prl.downstream.mdd.dataset import L2ArcticDataset
from s3prl.downstream.specaug import SpecAug
from s3prl.downstream.augment_utils.spectrogram import Spectrogram
from IPython.display import Audio, display
from torch.utils.data import RandomSampler, DataLoader, Subset
import einops
import torch
from speechbrain.lobes.augment import TimeDomainSpecAugment


In [9]:
dataset = L2ArcticDataset(
    'train',
    '/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0',
    '/home/xt0r3-user/cambridge/partii/dissertation/s3prl/s3prl/s3prl/downstream/mdd/data/'
)

Skipping malformatted TextGrid file: /home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0/YDCK/annotation/arctic_a0272.TextGrid
Skipping malformatted TextGrid file: /home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0/YDCK/annotation/arctic_a0209.TextGrid


In [10]:
def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()

#   num_frames = waveform.shape
#   if num_channels == 1:
  display(Audio(waveform, rate=sample_rate))
#   elif num_channels == 2:
#     display(Audio((waveform[0], waveform[1]), rate=sample_rate))
#   else:
#     raise ValueError("Waveform with more than 2 channels are not supported.")

In [11]:
play_audio(dataset[0][0], 44100)

In [23]:
augmentation = TimeDomainSpecAugment(
    speeds=[95, 100, 105],
    sample_rate=44100,
)
play_audio(augmentation(
    torch.nn.utils.rnn.pad_sequence([dataset[0][0]], batch_first=True),
    torch.tensor([len(dataset[0][0])], dtype=torch.long))[0], sample_rate=44100)

In [ ]:
SAMPLE_RATE = 44100
n_fft = 512
n_stft = int((n_fft // 2) + 1)
specaug = SpecAug(
    **{
        "adaptive": False,
        "adaptive_number_ratio": 0.04,
        "adaptive_size_ratio": 0.04,
        "max_n_time_masks": 20,
        "apply_time_warp": True,
        "apply_time_mask": True,
        "apply_freq_mask": True,
        "time_warp_window": 5,
        "time_mask_width_range": [0, 40],
        "freq_mask_width_range": [0, 50],
        "num_freq_mask": 4,
        "num_time_mask": 2,
    }
)
# transform = torchaudio.transforms.Spectrogram(n_fft=n_fft)
# inverse_transform = torchaudio.transforms.InverseSpectrogram(n_fft=n_fft)
# spectrogram = transform(dataset[0][0])
# print(spectrogram.dtype)
# augmented = specaug([spectrogram])
# display(augmented[0][0].size())
# display(augmented[0])
# raw = inverse_transform(spectrogram.type(torch.complex64))

device = 'cuda'

transform = Spectrogram(num_stft_frequencies=n_fft)
num_samples=10
subset = Subset(dataset, torch.arange(num_samples))
wavs = [data[0] for data in dataset]
aug_specs = [specaug(einops.rearrange(transform.wav_to_spec(wav), "h w complex -> complex h w"))[0] for wav in wavs]
# flat_spectrograms  = [spec[:, :, i] for spec in spectrograms for i in range(2)]
# aug_specs, _ = specaug(flat_spectrograms)
aug_wavs = [transform.spec_to_wav(torch.complex(specs[0], specs[1])) for specs in aug_specs]
# aug_wavs = []
# for specs in aug_specs:
#     aug_wavs.append(
#         transform.spec_to_wav(torch.complex(specs[0], specs[1])) 
#     )
# spectrogram = transformed_wavs = [
#     transform.spec_to_wav(specaug(transform.wav_to_spec([dataset[0][0], dataset[1][0]])
# print(spectrogram.size())
# augmented = [specaug([spectrogram[:, :, 0]])[0][0], specaug([spectrogram[:, :, 1]])[0][0]]
# display(augmented[0][0].size())
# display(augmented[0])
# raw = transform.spec_to_wav(torch.complex(spectrogram[:, :, 0], spectrogram[:, :, 1]))
# raw = transform.spec_to_wav(torch.complex(augmented[0], augmented[1]))

# display(raw.shape)
play_audio(aug_wavs[3].cpu(), 44100)

NameError: name 'torch' is not defined

In [ ]:
SAMPLE_RATE = 44100
n_fft = 512
n_stft = int((n_fft // 2) + 1)
specaug = SpecAug(
    **{
        "adaptive": False,
        "adaptive_number_ratio": 0.04,
        "adaptive_size_ratio": 0.04,
        "max_n_time_masks": 20,
        "apply_time_warp": True,
        "apply_time_mask": True,
        "apply_freq_mask": True,
        "time_warp_window": 5,
        "time_mask_width_range": [0, 40],
        "freq_mask_width_range": [0, 50],
        "num_freq_mask": 4,
        "num_time_mask": 2,
    }
)
# transform = torchaudio.transforms.Spectrogram(n_fft=n_fft)
# inverse_transform = torchaudio.transforms.InverseSpectrogram(n_fft=n_fft)
# spectrogram = transform(dataset[0][0])
# print(spectrogram.dtype)
# augmented = specaug([spectrogram])
# display(augmented[0][0].size())
# display(augmented[0])
# raw = inverse_transform(spectrogram.type(torch.complex64))

device = "cuda"

transform = Spectrogram(num_stft_frequencies=n_fft)
spectrogram = transform.wav_to_spec(dataset[0][0])
print(spectrogram.size())
augmented = [specaug([spectrogram[:, :, 0]])[0][0], specaug([spectrogram[:, :, 1]])[0][0]]
display(augmented[0][0].size())
display(augmented[0])
# raw = transform.spec_to_wav(torch.complex(spectrogram[:, :, 0], spectrogram[:, :, 1]))
raw = transform.spec_to_wav(torch.complex(augmented[0], augmented[1]))

# display(raw.shape)
play_audio(raw, 44100)

torch.Size([257, 1696, 2])


torch.Size([1696])

tensor([[ 1.6890e+00,  8.0134e-01, -4.4316e-01,  ...,  1.9210e+00,
          2.4674e+00,  3.5586e+00],
        [-1.0599e+00, -6.8823e-01,  7.2152e-01,  ..., -2.2980e+00,
         -1.0170e+00,  2.6528e+00],
        [-1.1883e+00,  7.9687e-01, -9.5308e-01,  ...,  1.2337e+00,
         -1.5089e+00,  1.5040e+00],
        ...,
        [-2.0601e-04,  1.0282e-02,  1.8358e-03,  ..., -7.6952e-03,
         -5.5027e-03, -5.4067e-05],
        [ 2.1992e-03, -5.4460e-05,  1.1225e-02,  ..., -7.4534e-03,
          3.5810e-03, -1.5751e-02],
        [ 3.6023e-03,  9.2963e-03,  5.7542e-03,  ...,  5.3479e-03,
          1.1652e-02,  4.6396e-03]])